# Task 1: Preprocess and Explore the Data

## Objective
Load, clean, and understand the historical financial data for TSLA, BND, and SPY. This analysis will inform our modeling decisions in the next tasks.

### Assets
- **TSLA (Tesla)**: High-growth stock, high risk.
- **BND (Vanguard Total Bond Market ETF)**: Bond market tracking, low risk, stability.
- **SPY (S&P 500 ETF)**: Broad market exposure, moderate risk.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set plot style
sns.set_theme(style="whitegrid")

## 1. Load and Clean the Data

The initial inspection showed that the CSVs have a "triple header" (Ticker name and empty Date labels in the first few rows). We need to filter those out and ensure numeric types.

In [ ]:
data_path = "../data/processed"
assets = ["TSLA", "BND", "SPY"]

def load_and_clean(ticker, path):
    file_path = os.path.join(path, f"{ticker}_historical_data.csv")
    # Load everything as strings initially to handle the triple header manually
    df = pd.read_csv(file_path)
    
    # Filter out redundant header rows
    df = df[~df['Price'].isin(['Ticker', 'Date'])].copy()
    
    # Rename columns
    df.rename(columns={'Price': 'Date'}, inplace=True)
    
    # Convert Date
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Convert others to float
    for col in ['Close', 'High', 'Low', 'Open', 'Volume']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    df.dropna(inplace=True)
    df.set_index('Date', inplace=True)
    df.sort_index(inplace=True)
    return df

dfs = {asset: load_and_clean(asset, data_path) for asset in assets}

for asset, df in dfs.items():
    print(f"{asset} loaded and cleaned. Shape: {df.shape}")

## 2. Basic Inspection (Post-Cleaning)

In [ ]:
for asset, df in dfs.items():
    print(f"\n--- {asset} Statistics ---")
    display(df.describe())